In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp


- convert an integer to bit string for bit string generation
- fix directions
- N x M matrix
- save lowest energies and configurations
- verify it works with -J (schrodingers cat)

Step 1: Define the Lattice Structure
- Lattice Shape (number indicates a dipole exists, 0 indicates no dipole)
- Spin states (each dipole will be represented by an entry in the lattice with a value of either +1 or -1)

Step 2: Hamiltonian Construction

- The hamiltonian describes the total energy of the system, accounting for interactions between neightboring spins. The enerrgy interaction between two neightboring spins is typically proportional to the product of their spin states (i.e. $H = -J\sum S_iS_j$, where $J$ is a coupling constant and $S_i$ and $S_j$ are the spin states of neighbors.)
- Once lattice is constructed, we need to filter out which dipoles are neighbors. For a 2D array, a dipole can interact with its left, right, top, below, top right, top left, bottom right, bottom left neighbors.

Step 3: Calculate total energy

- Use $H = -J\sum S_iS_j$ formula and sum up the energy contributions from every dipole in the lattice to determine the "total energy"

In [ ]:
# Step 1: Generate a random NxN bit string

np.random.seed(69420)

N = 3 # Lattice size
bit_string = np.random.choice([0, 1], size=(N * N))  # Random bit string of 0s and 1s

lattice = bit_string.reshape(N, N)
spin_lattice = np.where(lattice == 0, 1, -1) # Convert 0 to 1 (up spin) and 1 to -1 (down spin)

def hamiltonian(spin_lattice, J):

    total_energy = 0
    rows, cols = spin_lattice.shape

    for i in range(rows):
        for j in range(cols):
            if spin_lattice[i][j] != 0:  # Only consider if current index is a dipole
                
                neighbors = [
                    (i, j + 1),   # Right
                    (i + 1, j),   # Down
                    (i + 1, j + 1),  # Down-Right
                    (i + 1, j - 1)   # Down-Left
                ]
                
                for ni, nj in neighbors:
                    if 0 <= ni < rows and 0 <= nj < cols:  # Check boundaries
                        total_energy += -J * spin_lattice[i][j] * spin_lattice[ni][nj]

    return total_energy

J = 1  # Coupling constant
total_energy = hamiltonian(spin_lattice, J)

print("Bit String:\n", bit_string)
print("\nSpin Lattice:\n", spin_lattice)
print("\nTotal Energy:", total_energy)

def hamiltonian_boundary(spin_lattice, J):

    total_energy = 0
    rows, cols = spin_lattice.shape

    for i in range(rows):
        for j in range(cols):
            if spin_lattice[i][j] != 0:  # Only consider if current index is a dipole
                
                # Neighbors with periodic boundary conditions
                neighbors = [
                    ((i + 0) % rows, (j + 1) % cols),  # Right
                    ((i + 1) % rows, (j + 0) % cols),  # Down
                    ((i + 1) % rows, (j + 1) % cols),  # Down-Right
                    ((i + 1) % rows, (j - 1) % cols)   # Down-Left
                ]
                
                for ni, nj in neighbors:
                    total_energy += -J * spin_lattice[i][j] * spin_lattice[ni][nj]

    return total_energy

total_energy = hamiltonian_boundary(spin_lattice, J)

print("\nTotal Energy (periodic boundary) :", total_energy)

Bit String:
 [1 0 1 1 0 0 0 1 1]

Spin Lattice:
 [[-1  1 -1]
 [-1  1  1]
 [ 1 -1 -1]]

Total Energy: 6

Total Energy (periodic boundary) : 4


Suppose we are examining dipole at (2,2) in the lattice. It's right neighbour will be (2,3), however column 3 is out of bounds. If we plug into the formula, we get:
$$j_r = ( (2+1) \% 3) = 0$$

 so neighbour is at (2,0), which is what we're looking for.

In [6]:
def create_triangular_lattice(N):
    lattice = np.zeros((N, N))  # Start with all zeros
    
    # Fill a triangular pattern (values will be 1 or -1)
    for i in range(N):
        for j in range(i+1):  # Make a triangular pattern
            lattice[i, j] = 1
    return lattice

# Step 1: Generate a random NxN bit string
N = 5  # Lattice size (can change this value)
bit_string = np.random.choice([0, 1], size=(N * N))  # Random bit string of 0s and 1s

# Create a triangular lattice and convert to spin values
lattice = create_triangular_lattice(N)
spin_lattice = np.where(lattice == 1, 1, -1)  # Convert to 1 for up-spin, -1 for down-spin

# Step 2: Hamiltonian calculation with periodic boundary conditions
def hamiltonian_with_periodicity(spin_lattice, J):
    total_energy = 0
    rows, cols = spin_lattice.shape

    for i in range(rows):
        for j in range(cols):
            if spin_lattice[i][j] != 0:  # Only consider if current index is a dipole
                
                neighbors_1 = [
                    (i, (j + 1) % cols),  # Right (wrap around)
                    ((i + 1) % rows, j),  # Down (wrap around)
                ]

                neighbors_2 = [
                    ((i + 1) % rows, (j - 1) % cols),  # Bottom-left
                    ((i + 1) % rows, (j + 1) % cols)   # Bottom-right
                ]

                # First, process nearest-neighbor interactions
                for ni, nj in neighbors_1:
                    total_energy += -J * spin_lattice[i][j] * spin_lattice[ni][nj]  # Ferromagnetic
                
                # Then, process next-nearest-neighbor interactions
                for ni, nj in neighbors_2:
                    total_energy += J * spin_lattice[i][j] * spin_lattice[ni][nj]  # Competing interaction (could be ferromagnetic or anti-ferromagnetic)
    
    return total_energy

total_energy = hamiltonian_with_periodicity(spin_lattice, J)
# Output the results
print("Triangular Lattice:\n")
print(spin_lattice)
print("\nTotal Energy with Frustration:", total_energy)

Triangular Lattice:

[[ 1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1]
 [ 1  1  1 -1 -1]
 [ 1  1  1  1 -1]
 [ 1  1  1  1  1]]

Total Energy with Frustration: -12
